In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install surprise

In [2]:
df_dataset=pd.read_csv('C:\\Users\\vijeta\\Music\\ML-SimpliLearn-All-Files\\Amazon-Recommendation-project\\1567507480_amazonmoviesandtvratings\\Amazon-Ratings.csv')

In [3]:
df_dataset.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_dataset.shape

(4848, 207)

In [5]:
df_org=df_dataset.copy()
df_dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Movie1,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie2,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie3,1.0,2.000000,NaN,2.0,2.00,2.0,2.0,2.0
Movie4,2.0,5.000000,0.000000,5.0,5.00,5.0,5.0,5.0
Movie5,29.0,4.103448,1.496301,1.0,4.00,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...
Movie202,6.0,4.333333,1.632993,1.0,5.00,5.0,5.0,5.0
Movie203,1.0,3.000000,NaN,3.0,3.00,3.0,3.0,3.0
Movie204,8.0,4.375000,1.407886,1.0,4.75,5.0,5.0,5.0
Movie205,35.0,4.628571,0.910259,1.0,5.00,5.0,5.0,5.0


In [6]:
#Which movies have maximum views/ratings?
df_dataset.describe().T['count'].sort_values(ascending=False)[:1].to_frame()


,count
Movie127,2313.0


In [7]:
#Movie with highest Ratings
df_dataset.drop('user_id',axis=1).sum().sort_values(ascending=False)[:1].to_frame()

,0
Movie127,9511.0


In [8]:
#What is the average rating for each movie? Define the top 5 movies with the maximum ratings
df_dataset.drop('user_id',axis=1).mean().sort_values(ascending=False)[:5].to_frame()

,0
Movie1,5.0
Movie66,5.0
Movie76,5.0
Movie75,5.0
Movie74,5.0


In [9]:
#Define the top 5 movies with the least audience
df_dataset.describe().T['count'].sort_values(ascending=True)[:5].to_frame()

,count
Movie1,1.0
Movie71,1.0
Movie145,1.0
Movie69,1.0
Movie68,1.0


In [10]:
#Recommendation model
import surprise
from surprise import Reader
from surprise import accuracy
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import cross_validate

ModuleNotFoundError: No module named 'surprise'

In [12]:
df_melt = df_dataset.melt(id_vars = df_dataset.columns[0],value_vars=df_dataset.columns[1:],var_name="Movies",value_name="Rating")

In [13]:
df_melt

,user_id,Movies,Rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,NaN
2,A3LKP6WPMP9UKX,Movie1,NaN
3,AVIY68KEPQ5ZD,Movie1,NaN
4,A1CV1WROP5KTTW,Movie1,NaN
...,...,...,...
998683,A1IMQ9WMFYKWH5,Movie206,5.0
998684,A1KLIKPUF5E88I,Movie206,5.0
998685,A5HG6WFZLO10D,Movie206,5.0
998686,A3UU690TWXCG1X,Movie206,5.0


In [14]:
rd = Reader()
data = Dataset.load_from_df(df_melt.fillna(0),reader=rd)
data

In [15]:
trainset, testset = train_test_split(data,test_size=0.25)

In [16]:
#Using SVD (Singular Value Descomposition)
svd = SVD()
svd.fit(trainset)

In [17]:
pred = svd.test(testset)

In [18]:
accuracy.rmse(pred)

RMSE: 1.0261


1.0261477643037749

In [19]:
accuracy.mae(pred)

MAE:  1.0122


1.0121935833325506

In [20]:
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0264  1.0266  1.0258  1.0263  0.0004  
MAE (testset)     1.0121  1.0123  1.0120  1.0121  0.0001  
Fit time          35.81   36.14   36.10   36.02   0.15    
Test time         3.40    3.02    2.99    3.14    0.18    


{'test_rmse': array([1.02644208, 1.02660542, 1.0257933 ]),
 'test_mae': array([1.01208562, 1.01228049, 1.01200237]),
 'fit_time': (35.814096450805664, 36.13803815841675, 36.10470414161682),
 'test_time': (3.398491621017456, 3.019810676574707, 2.994333028793335)}

In [21]:
def repeat(ml_type,dframe):
    rd = Reader()
    data = Dataset.load_from_df(dframe,reader=rd)
    print(cross_validate(ml_type, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True))
    print("--"*15)
    usr_id = 'A3R5OBKS7OM2IR'
    mv = 'Movie1'
    r_u = 5.0
    print(ml_type.predict(usr_id,mv,r_ui = r_u,verbose=True))
    print("--"*15)

In [22]:
repeat(SVD(),df_melt.fillna(df_melt['Rating'].mean()))
#repeat(SVD(),df_melt.fillna(df_melt['Rating'].median()))

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.0865  0.0864  0.0856  0.0862  0.0004  
MAE (testset)     0.0100  0.0098  0.0098  0.0098  0.0001  
Fit time          35.79   35.98   36.08   35.95   0.12    
Test time         3.62    3.18    3.63    3.48    0.21    
{'test_rmse': array([0.08652968, 0.08643213, 0.0856383 ]), 'test_mae': array([0.00997113, 0.00975864, 0.0098112 ]), 'fit_time': (35.788992404937744, 35.9844970703125, 36.08207035064697), 'test_time': (3.6217849254608154, 3.1819674968719482, 3.626823663711548)}
------------------------------
user: A3R5OBKS7OM2IR item: Movie1     r_ui = 5.00   est = 4.40   {'was_impossible': False}
user: A3R5OBKS7OM2IR item: Movie1     r_ui = 5.00   est = 4.40   {'was_impossible': False}
------------------------------


In [23]:
#trying grid search and find optimum hyperparameter value for n_factors
from surprise.model_selection import GridSearchCV

In [24]:
param_grid = {'n_epochs':[20,30],
             'lr_all':[0.005,0.001],
             'n_factors':[50,100]}

In [ ]:
gs = GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
data1 = Dataset.load_from_df(df_melt.fillna(df_melt['Rating'].mean()),reader=rd)
gs.fit(data1)

In [ ]:
gs.best_score

In [ ]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])